# Data Cleaning

In [1]:
import pandas as pd

## Clean second layer info except broker/agencies

In [2]:
snd_df = pd.read_csv('unprocessed_sndlayer_info.csv')
snd_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971
0,32609kr/m²,15900kr/m²,34043kr/m²,26129kr/m²,34468kr/m²,19318kr/m²,19000kr/m²,31847kr/m²,16667kr/m²,28514kr/m²,...,63194kr/m²,44498kr/m²,44498kr/m²,64901kr/m²,40204kr/m²,63368kr/m²,63905kr/m²,69410kr/m²,69410kr/m²,85124kr/m²
1,695000kr,795000kr,725000kr,795000kr,750000kr,850000kr,975000kr,950000kr,1050000kr,895000kr,...,7995000kr,9300000kr,9300000kr,6995000kr,8500000kr,9500000kr,9500000kr,10000000kr,10000000kr,11600000kr
2,+55000 kr (+8%),NaN,+75000 kr (+10%),+15000 kr (+2%),+60000 kr (+8%),NaN,-25000 kr (-3%),+50000 kr (+5%),NaN,+160000 kr (+18%),...,"+1,11milj. kr (+14%)",NaN,NaN,"+2,81milj. kr (+40%)","+1,35milj. kr (+16%)",+1milj. kr (+11%),"+1,3milj. kr (+14%)","+1,05milj. kr (+11%)","+1,05milj. kr (+11%)",+62000 kr (+1%)
3,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet,...,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet,Lägenhet
4,Bostadsrätt,Bostadsrätt,Bostadsrätt,Bostadsrätt,Bostadsrätt,Bostadsrätt,Bostadsrätt,Bostadsrätt,Bostadsrätt,Bostadsrätt,...,Bostadsrätt,Bostadsrätt,Bostadsrätt,Andel i bostadsförening,Bostadsrätt,Bostadsrätt,Bostadsrätt,Bostadsrätt,Bostadsrätt,Bostadsrätt
5,1 rum,2 rum,1 rum,1 rum,1 rum,2 rum,1 rum,"1,5 rum",2 rum,1 rum,...,5 rum,6 rum,6 rum,5 rum,6 rum,6 rum,5 rum,5 rum,5 rum,4 rum
6,23 m²,50 m²,"23,5 m²",31 m²,"23,5 m²",44 m²,50 m²,"31,4 m²",63 m²,37 m²,...,144 m²,209 m²,209 m²,151 m²,245 m²,"165,7 m²",169 m²,"159,2 m²","159,2 m²",137 m²
7,Nej,"2 av 2, hiss finns ej","2 av 2, hiss finns ej",2 av 2,"1 av 2, hiss finns ej","2 av 2, hiss finns ej",Ja,Nej,2 av 2,"1 av 3, hiss finns ej",...,Nej,Ja,10626kr/mån,Nej,Ja,Ja,Ja,Ja,Ja,6909kr/mån
8,"1 av 3, hiss finns ej",2004,1957,2018,1957,1953,"2 av 2, hiss finns ej","1 av 8, hiss finns",2004,1971,...,Ja,10626kr/mån,NaN,"1 av 3, hiss finns ej","2 av 5, hiss finns","5 av 5, hiss finns","2 av 4, hiss finns ej","3 av 4, hiss finns ej","3 av 4, hiss finns ej",Brf Kulturkvarteret i Lund
9,1956,4011kr/mån,1836kr/mån,1770kr/mån,1786kr/mån,2967kr/mån,1971,1964,4987kr/mån,2228kr/mån,...,"1 av 2, hiss finns ej",NaN,NaN,1903,1890,8008kr/mån,1904,1903,1903,NaN


### Asking prices

In [3]:
asking_price=snd_df.iloc[1].str.replace('kr', '').astype(float)
asking_price.head()

0    695000.0
1    795000.0
2    725000.0
3    795000.0
4    750000.0
Name: 1, dtype: float64

### Building year

In [6]:
build_year = [None for _ in range(1972)]

for i in range(1972):
    snd_df_col = snd_df[str(i)]
    for element in snd_df_col:
        # here we need to check if the element is string because we set asking_price data type
        # to be float which does not have length. 
        if type(element)==str and len(element) == 4:
            build_year[i] = int(element)
            
build_year[0:6]

[1956, 2004, 1957, 2018, 1957, 1953]

### Wether there is a balcony/patio

#### Let's check how many sold apartments provide the info of balcony/patio

In [9]:
is_balcony_count = [None for _ in range(1972)]
for i in range(1972):
    snd_df_col = snd_df[str(i)]
    Nej_count = list(snd_df_col).count('Nej')
    Ja_count = list(snd_df_col).count('Ja')
    is_balcony_count[i] = Nej_count + Ja_count

In [10]:
# 242 apartments do not provide this info
is_balcony_count.count(0)

242

In [11]:
# 1520 apartments only provide whether there is a balcony
is_balcony_count.count(1)

1520

In [12]:
# 210 apartments provide both balcony and patio info
is_balcony_count.count(2)

210

#### Corresponding indices

In [13]:
balcony_index = [i for i, e in enumerate(is_balcony_count) if e == 1]
balcony_patio_index = [i for i, e in enumerate(is_balcony_count) if e == 2]

#### Now we can get the info from the items with given indices

In [14]:
is_balcony = [None for _ in range(1972)]
is_patio = [None for _ in range(1972)]

In [18]:
# For the links which only provide balcony info, there is either 'Ja' or 'Nej'

for ind in balcony_index:
    snd_df_col = snd_df[str(ind)]
    for element in snd_df_col:
        if element == 'Ja' or element == 'Nej':
            is_balcony[ind] = element

In [19]:
# For the links provide both the balcony and patio info, the 7th element gives balcony and 8th element gives patio

for ind in balcony_patio_index:
    snd_df_col = snd_df[str(ind)]
    is_balcony[ind] = snd_df_col[7]
    is_patio[ind] = snd_df_col[8]

In [20]:
is_balcony[0:10]

['Nej', None, None, None, None, None, 'Ja', 'Nej', None, None]

In [22]:
is_patio[5:15]

[None, None, None, None, None, 'Ja', None, None, None, None]

### Total number of building floors/Apartment floor number/If elevator is available. 

In [25]:
floor_elevator = [None for _ in range(1972)]

for i in range(1972):
    snd_df_col = snd_df[str(i)]
    for element in snd_df_col:
        if type(element)==str and element.count('av') == 1:
            floor_elevator[i] = element
            
floor_elevator[0:5]

['1 av 3, hiss finns ej',
 '2 av 2, hiss finns ej',
 '2 av 2, hiss finns ej',
 '2 av 2',
 '1 av 2, hiss finns ej']

#### From the above output, we find that building floors/Apartment floor number is separated by the string 'av', and is_elevator is obtained simply by first replacing 'hiss finns ej' as 'No' and 'hiss finns' as 'Yes'.

In [26]:
floor_number = [None for _ in range(1972)]
total_floor = [None for _ in range(1972)]
is_elevator = [None for _ in range(1972)]

In [27]:
for i in range(1972):
    if floor_elevator[i]:
        element = floor_elevator[i].split(', ')
        
        if len(element) == 2:
            is_elevator[i] = element[1]
            floor_info = element[0].split('av')
            floor_number[i] = floor_info[0].strip()
            total_floor[i] = floor_info[1].strip()
        elif len(element) == 1:
            floor_info = element[0].split('av')
            floor_number[i] = floor_info[0].strip()
            total_floor[i] = floor_info[1].strip()

In [28]:
floor_number[0:6]

['1', '2', '2', '2', '1', '2']

In [29]:
total_floor[0:6]

['3', '2', '2', '2', '2', '2']

In [31]:
is_elevator = list(pd.Series(is_elevator).replace('hiss finns ej','No').replace('hiss finns','Yes'))
is_elevator[0:10]

['No', 'No', 'No', None, 'No', 'No', 'No', 'Yes', None, 'No']

### Merge into one CSV file

In [ ]:
d = {'Average': average, 'Asking price': asking_price,'Balcony': is_balcony, 'Patio': is_patio,'Build year': build_year, 'Floor number': floor_number, 'Total floor': total_floor, 'Elevator': is_elevator}
snd_layer_df = pd.DataFrame(data=d)
snd_layer_df.to_csv('snd_layer_df_info.csv', index=False)

## Clean first layer info

In [152]:
df = pd.read_csv('hemnet.csv')
df.head()

,Addresses,Types,area (m²),# of rooms,Monthly Fees (Kr),Sold Dates,Links,Prices (tKr)
0,Flormansgatan 2A,Lägenhet,43,1.5,2767.0,30 september 2021,"https://www.hemnet.se/salda/lagenhet-1,5rum-ce...",2370
1,Kastanjegatan 19F,Lägenhet,34,2.0,2415.0,30 september 2021,https://www.hemnet.se/salda/lagenhet-2rum-jarn...,1745
2,Karl XI gatan 47,Lägenhet,"87,4",3.0,5787.0,30 september 2021,https://www.hemnet.se/salda/lagenhet-3rum-lund...,4700
3,Äspet 163,Villa,158 + 22,8.0,NaN,30 september 2021,https://www.hemnet.se/salda/villa-8rum-lunds-k...,5350
4,Margaretavägen 3K,Lägenhet,78,3.0,4584.0,30 september 2021,https://www.hemnet.se/salda/lagenhet-3rum-moll...,2750


### Separate Apartment (Lägenhet) from other housing types

In [153]:
apart_df = df[df['Types'] == 'Lägenhet']
apart_df.head()

,Addresses,Types,area (m²),# of rooms,Monthly Fees (Kr),Sold Dates,Links,Prices (tKr)
0,Flormansgatan 2A,Lägenhet,43,1.5,2767.0,30 september 2021,"https://www.hemnet.se/salda/lagenhet-1,5rum-ce...",2370
1,Kastanjegatan 19F,Lägenhet,34,2.0,2415.0,30 september 2021,https://www.hemnet.se/salda/lagenhet-2rum-jarn...,1745
2,Karl XI gatan 47,Lägenhet,"87,4",3.0,5787.0,30 september 2021,https://www.hemnet.se/salda/lagenhet-3rum-lund...,4700
4,Margaretavägen 3K,Lägenhet,78,3.0,4584.0,30 september 2021,https://www.hemnet.se/salda/lagenhet-3rum-moll...,2750
5,Qvantenborgsvägen 4B,Lägenhet,59,2.0,3125.0,29 september 2021,https://www.hemnet.se/salda/lagenhet-2rum-kobj...,2250


In [155]:
apart_df.shape

(1972, 8)

### Let's first clean the data in the column 'area'

In [121]:
area = apart_df['area (m²)']
area.head()

0      43
1      34
2    87,4
4      78
5      59
Name: area (m²), dtype: object

In [122]:
# We first clean the values contain '+' sign by removing the number after it. 
# By comparing the sold price and price/m², seems these numbers are not counted. 

irregular_values = apart_df[apart_df['area (m²)'].str.contains('+', regex=False)]['area (m²)']
irregular_values.head()

69       60 + 20
91       44 + 20
154    48,4 + 20
485    75,5 + 21
958      89 + 50
Name: area (m²), dtype: object

In [123]:
regular_values = irregular_values.str.split('+').str[0]
regular_values.head()

69       60 
91       44 
154    48,4 
485    75,5 
958      89 
Name: area (m²), dtype: object

In [124]:
irregular_index = apart_df[apart_df['area (m²)'].str.contains('+', regex=False)].index.values

for ind in irregular_index:
    area = area.replace(area[ind], regular_values[ind])

In [125]:
area[91]

'44 '

In [126]:
# We also replace comma with period.

area = area.str.replace(',','.')
area.head()

0      43
1      34
2    87.4
4      78
5      59
Name: area (m²), dtype: object

In [160]:
# Now replace the column with the cleaned values.

pd.options.mode.chained_assignment = None  # default='warn'
apart_df['area (m²)'] = area
# apart_df = apart_df[apart_df['area (m²)'] == area] # sth wrong with this line of code which changes apart_df shape.
apart_df.head()

,Addresses,Types,area (m²),# of rooms,Monthly Fees (Kr),Sold Dates,Links,Prices (tKr)
0,Flormansgatan 2A,Lägenhet,43,1.5,2767.0,30 september 2021,"https://www.hemnet.se/salda/lagenhet-1,5rum-ce...",2370
1,Kastanjegatan 19F,Lägenhet,34,2.0,2415.0,30 september 2021,https://www.hemnet.se/salda/lagenhet-2rum-jarn...,1745
2,Karl XI gatan 47,Lägenhet,87.4,3.0,5787.0,30 september 2021,https://www.hemnet.se/salda/lagenhet-3rum-lund...,4700
4,Margaretavägen 3K,Lägenhet,78,3.0,4584.0,30 september 2021,https://www.hemnet.se/salda/lagenhet-3rum-moll...,2750
5,Qvantenborgsvägen 4B,Lägenhet,59,2.0,3125.0,29 september 2021,https://www.hemnet.se/salda/lagenhet-2rum-kobj...,2250


In [161]:
apart_df.shape

(1972, 8)

### Change the format of Sold Dates

In [162]:
Dates = apart_df['Sold Dates']
Dates=Dates.str.replace(' januari ','/01/').str.replace(' februari ','/02/').str.replace(' mars ','/03/').str.replace(' april ','/04/').str.replace(' maj ','/05/').str.replace(' juni ','/06/').str.replace(' juli ','/07/').str.replace(' augusti ','/08/').str.replace(' september ','/09/').str.replace(' oktober ','/10/').str.replace(' november ','/11/').str.replace(' december ','/12/')
Dates.head()

0    30/09/2021
1    30/09/2021
2    30/09/2021
4    30/09/2021
5    29/09/2021
Name: Sold Dates, dtype: object

In [163]:
Dates = pd.to_datetime(Dates)
apart_df['Sold Dates'] = Dates.values
apart_df.head()

,Addresses,Types,area (m²),# of rooms,Monthly Fees (Kr),Sold Dates,Links,Prices (tKr)
0,Flormansgatan 2A,Lägenhet,43,1.5,2767.0,2021-09-30,"https://www.hemnet.se/salda/lagenhet-1,5rum-ce...",2370
1,Kastanjegatan 19F,Lägenhet,34,2.0,2415.0,2021-09-30,https://www.hemnet.se/salda/lagenhet-2rum-jarn...,1745
2,Karl XI gatan 47,Lägenhet,87.4,3.0,5787.0,2021-09-30,https://www.hemnet.se/salda/lagenhet-3rum-lund...,4700
4,Margaretavägen 3K,Lägenhet,78,3.0,4584.0,2021-09-30,https://www.hemnet.se/salda/lagenhet-3rum-moll...,2750
5,Qvantenborgsvägen 4B,Lägenhet,59,2.0,3125.0,2021-09-29,https://www.hemnet.se/salda/lagenhet-2rum-kobj...,2250


In [164]:
apart_df.shape

(1972, 8)

### Drop the columns 'Types' and 'Links' as they are not needed any more

In [165]:
apart_df = apart_df.drop(columns='Types')

In [166]:
apart_df = apart_df.drop(columns='Links')

In [167]:
apart_df.head()

,Addresses,area (m²),# of rooms,Monthly Fees (Kr),Sold Dates,Prices (tKr)
0,Flormansgatan 2A,43,1.5,2767.0,2021-09-30,2370
1,Kastanjegatan 19F,34,2.0,2415.0,2021-09-30,1745
2,Karl XI gatan 47,87.4,3.0,5787.0,2021-09-30,4700
4,Margaretavägen 3K,78,3.0,4584.0,2021-09-30,2750
5,Qvantenborgsvägen 4B,59,2.0,3125.0,2021-09-29,2250


### Reorder the rows as the increasing sold prices

In [168]:
apart_df = apart_df.sort_values(['Prices (tKr)'], ascending=1)
apart_df.head()

,Addresses,area (m²),# of rooms,Monthly Fees (Kr),Sold Dates,Prices (tKr)
565,Veberödsvägen 22C,23,1.0,1287.0,2021-04-09,750
319,Idalavägen 47 f,50,2.0,4011.0,2020-10-21,795
2007,Allégatan 3F,23.5,1.0,1836.0,2020-11-15,800
259,Horstgatan 4H,31,1.0,1770.0,2020-10-29,810
2411,Allégatan 3F,23.5,1.0,1786.0,2021-03-01,810


In [169]:
apart_df.shape

(1972, 6)

### Drop the index and add other features from second layer info

In [170]:
new_apart_df = apart_df.reset_index()

In [171]:
new_apart_df.head(3)

,index,Addresses,area (m²),# of rooms,Monthly Fees (Kr),Sold Dates,Prices (tKr)
0,565,Veberödsvägen 22C,23,1.0,1287.0,2021-04-09,750
1,319,Idalavägen 47 f,50,2.0,4011.0,2020-10-21,795
2,2007,Allégatan 3F,23.5,1.0,1836.0,2020-11-15,800


In [172]:
agent_df = pd.read_csv('agent.csv')
snd_info_df = pd.read_csv('snd_layer_df_info.csv')

In [213]:
# Join 3 dataframes

new_df = pd.concat([new_apart_df, snd_info_df, agent_df], axis=1)

In [214]:
# Get rid of NaN 

new_df = new_df.fillna('')
new_df.head()

,index,Addresses,area (m²),# of rooms,Monthly Fees (Kr),Sold Dates,Prices (tKr),Average,Asking price,Balcony,Patio,Build year,Floor number,Total floor,Elevator,Agents,Agencies
0,565,Veberödsvägen 22C,23,1.0,1287.0,2021-04-09,750,32609.0,695000,Nej,,1956,1,3,No,Karin Ekström,Erik Olsson Fastighetsförmedling
1,319,Idalavägen 47 f,50,2.0,4011.0,2020-10-21,795,15900.0,795000,,,2004,2,2,No,Rickard Saltin,Fastighetsbyrån Lund
2,2007,Allégatan 3F,23.5,1.0,1836.0,2020-11-15,800,34043.0,725000,,,1957,2,2,No,,Fastighetsbyrån Lund
3,259,Horstgatan 4H,31,1.0,1770.0,2020-10-29,810,26129.0,795000,,,2018,2,2,,,Svensk Fastighetsförmedling Lund
4,2411,Allégatan 3F,23.5,1.0,1786.0,2021-03-01,810,34468.0,750000,,,1957,1,2,No,,Fastighetsbyrån Lund


In [99]:
# 'Average': average, 'Asking price': asking_price,'Balcony': is_balcony, 
#'Patio': is_patio,'Build year': build_year, 'Floor number': floor_number, 
#'Total floor': total_floor, 'Elevator': is_elevator}

### Change data type/reorder columns

#### We change the column 'asking price' also to 'asking price (tkr)'

In [210]:
# Two apartments do not have asking prices

none_index=[i for i,v in enumerate(new_df['Asking price']) if v == '']
none_index

[1399, 1473]

In [211]:
# We replace these 2 values to 0. Remember to drop these 2 values in analysis using this feature!

ask_price = new_df['Asking price'].replace('',0)
ask_price_tkr = ask_price/1000
ask_price_tkr = ask_price_tkr.astype(int)

In [215]:
new_df = new_df.drop(columns='index')
new_df = new_df.drop(columns='Asking price')
new_df['Asking (tKr)'] = ask_price_tkr

In [221]:
apartment_df = new_df.reindex(columns=['Addresses', 'area (m²)', '# of rooms', 'Balcony', 'Patio', 'Elevator', 'Floor number', 'Total floor', 'Monthly Fees (Kr)', 'Build year', 'Asking (tKr)', 'Prices (tKr)', 'Average', 'Agents', 'Agencies', 'Sold Dates'])

In [222]:
apartment_df.head(3)

,Addresses,area (m²),# of rooms,Balcony,Patio,Elevator,Floor number,Total floor,Monthly Fees (Kr),Build year,Asking (tKr),Prices (tKr),Average,Agents,Agencies,Sold Dates
0,Veberödsvägen 22C,23,1.0,Nej,,No,1,3,1287.0,1956,695,750,32609.0,Karin Ekström,Erik Olsson Fastighetsförmedling,2021-04-09
1,Idalavägen 47 f,50,2.0,,,No,2,2,4011.0,2004,795,795,15900.0,Rickard Saltin,Fastighetsbyrån Lund,2020-10-21
2,Allégatan 3F,23.5,1.0,,,No,2,2,1836.0,1957,725,800,34043.0,,Fastighetsbyrån Lund,2020-11-15


### Save as a CSV file

In [223]:
apartment_df.to_csv('apartment_df.csv')